# Traffic Accident Risk Prediction using ML/DL
This notebook includes data preprocessing, EDA, and modeling steps using Random Forest, XGBoost, and a simulated CNN setup.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import shap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the dataset (replace with actual path)
df = pd.read_csv('your_dataset.csv')
df.head()

## Data Preprocessing

In [ ]:
# Sample preprocessing (customize based on your dataset)
categorical = ['Weather', 'Road_Type', 'Vehicle_Type', 'Time_of_Day']
numerical = ['Speed', 'Driver_Age']

# Imputation and Encoding
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('impute', SimpleImputer(strategy='mean')),
        ('scale', StandardScaler())
    ]), numerical),
    ('cat', Pipeline([
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('encode', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical)
])

X = df[categorical + numerical]
y = df['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

## Exploratory Data Analysis

In [ ]:
sns.countplot(data=df, x='Severity')
plt.title('Severity Distribution')
plt.show()
sns.boxplot(data=df, x='Severity', y='Speed')
plt.title('Speed vs Severity')
plt.show()

## Random Forest Model

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_transformed, y_train)
y_pred_rf = rf.predict(X_test_transformed)
print(classification_report(y_test, y_pred_rf))

## XGBoost Model

In [ ]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train_transformed, y_train)
y_pred_xgb = xgb.predict(X_test_transformed)
print(classification_report(y_test, y_pred_xgb))

## SHAP Analysis for XGBoost

In [ ]:
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(X_test_transformed)
shap.summary_plot(shap_values, X_test_transformed, plot_type='bar')